In [1]:
import cv2
import numpy as np
import glob

k = np.zeros((4,4), 'float')
for i in range(4):
    k[i,i] = 1

temp_label = np.zeros((1,4),'float')
image_array1 = np.zeros((1,38400),'float')
label_array1 = np.zeros((1,4),'float')
'''
left    = [1 0 0 0]
forward = [0 1 0 0]
right   = [0 0 1 0]
reverse = [0 0 0 1]
'''

count = 0

for img in glob.glob("left/*.png"):
    #print(type(img))
    image = cv2.imread(img,0)
    #cv2.imshow('img',image)
    roi = image[120:240, :]
    temp_array1 = roi.reshape(1,38400).astype(np.float32)
    image_array1 = np.vstack((image_array1, temp_array1))
    label_array1 = np.vstack((label_array1, k[0]))
    count+=1
print('left done')
print(count)

count = 0

for img in glob.glob("right/*.png"):
    image = cv2.imread(img,0)
    #cv2.imshow('img',image)
    roi = image[120:240, :]
    temp_array1 = roi.reshape(1,38400).astype(np.float32)
    image_array1 = np.vstack((image_array1, temp_array1))
    label_array1 = np.vstack((label_array1, k[2]))
    count+=1
print('right done')
print(count)

count =0

for img in glob.glob("forward/*.png"):
    image = cv2.imread(img,0)
    #cv2.imshow('img',image)
    roi = image[120:240, :]
    temp_array1 = roi.reshape(1,38400).astype(np.float32)
    image_array1 = np.vstack((image_array1, temp_array1))
    label_array1 = np.vstack((label_array1, k[1]))
    count+=1
print('forward done')
print(count)

for img in glob.glob("reverse/*.png"):
    image = cv2.imread(img,0)
    #cv2.imshow('img',image)
    roi = image[120:240, :]
    temp_array1 = roi.reshape(1,38400).astype(np.float32)
    image_array1 = np.vstack((image_array1, temp_array1))
    label_array1 = np.vstack((label_array1, k[3]))
    count+=1
print('reverse done')
print(count)
print(label_array1)
print(image_array1)

train = image_array1[1:, :]
train_labels = label_array1[1:, :]
# save training data as a numpy file
np.savez('training_data_temp/test09.npz', train=train, train_labels=train_labels)
print('npz file saved!')

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [1]:
import cv2
import numpy as np
import glob

print('Loading training data...')
e0 = cv2.getTickCount()

# load training data
image_array = np.zeros((1, 38400))
label_array = np.zeros((1, 4), 'float')
training_data = glob.glob('training_data_temp/test09.npz')

for single_npz in training_data:
    with np.load(single_npz) as data:
        print (data.files)
        train_temp = data['train']
        train_labels_temp = data['train_labels']
        print (train_temp.shape)
        print (train_labels_temp.shape)
    image_array = np.vstack((image_array, train_temp))
    label_array = np.vstack((label_array, train_labels_temp))

train = image_array[1:, :]
train_labels = label_array[1:, :]
print (train.shape)
print (train_labels.shape)

e00 = cv2.getTickCount()
time0 = (e00 - e0)/ cv2.getTickFrequency()
print('Loading image duration:', time0)

# set start time
e1 = cv2.getTickCount()

# create MLP
layer_sizes = np.int32([38400,4,2,8,8,4])
model = cv2.ml.ANN_MLP_create()
model.setLayerSizes(layer_sizes)
#criteria = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 500, 0.0001)
#criteria2 = (cv2.TERM_CRITERIA_COUNT, 100, 0.001)
#params = dict(term_crit = criteria,
               #train_method = cv2.ml.ANN_MLP_BACKPROP,
               #bp_dw_scale = 0.001,
               #bp_moment_scale = 0.0 )
model.setTrainMethod(cv2.ml.ANN_MLP_BACKPROP)
model.setActivationFunction(cv2.ml.ANN_MLP_SIGMOID_SYM, 2, 1)
model.setBackpropWeightScale(0.001)

print('Training MLP ...')
num_iter = model.train(np.float32(train), cv2.ml.ROW_SAMPLE, np.float32(train_labels))

# set end time
e2 = cv2.getTickCount()
time = (e2 - e1)/cv2.getTickFrequency()
print('Training duration:', time)

# save param
model.save('mlp_xml/mlp.xml')

#print('Ran for %d iterations' % num_iter)

ret, resp = model.predict(train)
prediction = resp.argmax(-1)
#print('Prediction:', prediction)
true_labels = train_labels.argmax(-1)
#print('True labels:', true_labels)

print('Testing...')
train_rate = np.mean(prediction == true_labels)
print(prediction)
print(true_labels)
print ('Train rate: %f:' % (train_rate*100))

Loading training data...
['train', 'train_labels']
(245, 38400)
(245, 4)
(245, 38400)
(245, 4)
Loading image duration: 0.9928586243689447
Training MLP ...
Training duration: 9.158964994472587
Testing...
[0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1